In [9]:
import pyomo.environ as pe
import pyomo.opt as po


## Charter flights

Cristina Hernández, Macarena Vargas, Guillermo Ruiz, Beatriz Jiménez. 3ºIMAT B


Resumen del enunciado: 

Una compañía tiene 5 aviones de cada tipo (A,B,C). Cada tipo admite 80, 68 y 55 pasajeros. 
Quiere encontrar la mejor combinación de estos tipos para llevar a 372 pasajeros para tener el minimo coste . 
Tenemos una tabla del coste de cada tipo de avion, cuanto cuesta usar de 1-5 aviones. 
Además, tenemos un coste fijo de 6k € para cada tipo de avión que se usa. 





#### Creamos el modelo 

In [10]:
modelo = pe.ConcreteModel()


#### Sets 

${i}$: tipo de avión

${j}$: numero de aviones 



In [11]:
modelo.aviones= pe.Set(initialize=['A', 'B', 'C'])
modelo.numero_aviones = pe.Set(initialize=[1, 2, 3, 4, 5])


#### Parámetros 
$Q_{i}$: capacidad del avión i (número de pasajeros máximo)



In [12]:
modelo.capacidad= pe.Param(modelo.aviones, initialize={'A': 80, 'B':68, 'C': 55})

$FC$: costes fijos [en miles de €]



In [13]:
modelo.coste_fijo = pe.Param(initialize=6)

$C_{ij}$ : coste de usar j aviones del tipo i

In [14]:
costes = {('A', 1): 11, ('A', 2): 20, ('A', 3): 30, ('A', 4): 40, ('A', 5): 50,
            ('B', 1): 9, ('B', 2): 17, ('B', 3): 24, ('B', 4): 34, ('B', 5): 45,
            ('C', 1): 8, ('C', 2): 15, ('C', 3): 21, ('C', 4): 26, ('C', 5): 31}

modelo.costes = pe.Param(modelo.aviones, modelo.numero_aviones, initialize=costes)

#### Variables

$x_{ij}$ : se usan j aviones del tipo i [bool]. 

In [15]:
modelo.x = pe.Var(modelo.aviones, modelo.numero_aviones, domain=pe.Binary)

#### Función objetivo 

Queremos minimizar los costes. 
Para cada tipo de avión (i) y cada numero de avion j, si se elije esa combinación (xij=1), 
entonces se le suman los costes. los costes son los fijos (FC) + los variables (el coste de esa celda en la matriz de costes C)


min $\sum_{ij} (C_{ij} + FC) * x_{ij} $

In [ ]:
def funcion_objetivo(modelo):
    fc = modelo.coste_fijo

    return sum((modelo.costes[i,j] + fc) * modelo.x[i,j] for i in modelo.aviones for j in modelo.numero_aviones)

modelo.coste = pe.Objective(rule=funcion_objetivo, sense=pe.minimize)


#### Constrains 

Restricción 1) El número de pasajeros debe ser igual o superior a 372 
Restricción 2) Para cada avión i, solo puede haber una xij que tenga un 1. 
